# Human in the loop

In [13]:
from dotenv import load_dotenv

_ = load_dotenv()

In [14]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [15]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [16]:
tool = TavilySearchResults(max_results=2)

In [17]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"] #INTERRUPT IS USEFUL FOR THE USER TO INTERACT. 
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [18]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [19]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='e24f5b5b-29f6-4de4-9d67-9e99c5547838'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3JYAu0DuEHAc29H4hRUCyJRF', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d87dbdbe-304d-414c-b563-f314d13056a1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_3JYAu0DuEHAc29H4hRUCyJRF'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3JYAu0DuEHAc29H4hRUCyJRF', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name

In [20]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='e24f5b5b-29f6-4de4-9d67-9e99c5547838'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3JYAu0DuEHAc29H4hRUCyJRF', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d87dbdbe-304d-414c-b563-f314d13056a1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_3JYAu0DuEHAc29H4hRUCyJRF'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef3df77-d02b-6e59-8001-cc44b26f57e7'}}, metadata={'source': 'loop', 'step': 1, 'wri

In [21]:
abot.graph.get_state(thread).next

('action',)

In [22]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_3JYAu0DuEHAc29H4hRUCyJRF'}
Back to the model!
{'messages': [ToolMessage(content="[{'url': 'https://www.accuweather.com/en/us/san-francisco/94103/september-weather/347629', 'content': 'Avg. 71°. 55°. Temperature Graph. °F.Missing:  09/07/2024'}, {'url': 'https://www.weather.com/weather/tenday/l/94110:4:US', 'content': 'tenDayWeather-San Francisco, CA. asOfTime. Today. 70°/55°. 4%. Sun 07 | day. 70°. 4%. W 16 mph. Plenty of sunshine. High near 70F. Winds W at 10 to 20 mph.'}]", name='tavily_search_results_json', tool_call_id='call_3JYAu0DuEHAc29H4hRUCyJRF')]}
{'messages': [HumanMessage(content='Whats the weather in SF?', id='e24f5b5b-29f6-4de4-9d67-9e99c5547838'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3JYAu0DuEHAc29H4hRUCyJRF', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}

In [23]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='e24f5b5b-29f6-4de4-9d67-9e99c5547838'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3JYAu0DuEHAc29H4hRUCyJRF', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d87dbdbe-304d-414c-b563-f314d13056a1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_3JYAu0DuEHAc29H4hRUCyJRF'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173}), ToolMessage(content="[{'url': 'https://www.accuweather.com/en/us/san-francisco/94103/september-weather/347629', 'content': 'Avg. 71°. 55°. Temperature Graph. °F.

In [24]:
abot.graph.get_state(thread).next

()